# Ejemplo Completo de Decisión Estática Bajo Incertidumbre

## 1. Introducción

A continuación se presenta un ejemplo completo de cómo mapear y resolver un problema de decisión estática bajo incertidumbre con nuevas consideraciones:

* Distintas **universidades** (ITAM vs. Ibero) que influyen la probabilidad de tener un profesor estricto
* El factor de haber **estudiado** o ser buen alumno (modifica la utilidad de no hacer trampa)
* **Señales** (proxy) sobre la severidad del profesor

Imagina que eres estudiante y tienes un **examen** en puerta. Te planteas la decisión de **hacer trampa** o **no** hacerlo. Hay múltiples factores que influyen:

1. **Universidad** donde estudias (ITAM vs. Ibero):
   * En el **ITAM** hay fama de profesores **más estrictos**
   * En la **Ibero** podría haber más profesores relajados ("barcos")

2. **Nivel de estudio o preparación**:
   * Si **estudiaste** o eres naturalmente "buen alumno", te irá razonablemente bien sin hacer trampa
   * Si **no** estudiaste, tu calificación sin trampa puede ser menor

3. **Señales** (proxy) antes del examen:
   * El profesor publicó un correo amenazante (alta probabilidad de ser estricto)
   * El profesor es "relajado" en clase y no vigila mucho
   * No tienes señales claras

Bajo este entorno de incertidumbre, deseas **maximizar** tu **utilidad esperada** (o minimizar consecuencias negativas).

## 2. Mapeo del Problema

Siguiendo la teoría de la **decisión estática bajo incertidumbre**, definimos los elementos:

### 2.1 Decisiones ($D$)

* $d_0$: **No hacer trampa**
* $d_1$: **Hacer trampa**

### 2.2 Estados de la Naturaleza ($\Omega$)

Desconocidos antes del examen:
* $\omega_0$: Profesor "barco" (muy permisivo)
* $\omega_1$: Profesor "normal" (moderadamente vigilante)
* $\omega_2$: Profesor "muy estricto" (alto riesgo de descubrir trampas)

### 2.3 Información Parcial (proxy)

* $z_1$: "El profesor se ve **relajado**" (comentarios, actitud, no revisa tareas a detalle, etc.)
* $z_2$: "El profesor **amenaza** con sanciones, menciona software antiplagio, etc."
* $z_0$: "No hay señales claras"

Estas señales no son 100% confiables, pero permiten actualizar (vía Bayes) la distribución sobre $\omega$.

### 2.4 Universidad y "Haber Estudiado"

* **Universidad** = $\{\text{ITAM}, \text{Ibero}\}$
  * Impacta la **probabilidad inicial** de cada tipo de profesor ($\omega$)
* **Estudié (S/N)**: Afecta la **utilidad** obtenida si decides NO hacer trampa

### 2.5 Función de Utilidad $U(\omega, d)$

1. **No hacer trampa** ($d_0$):
   * Si **estudié**, la utilidad de no hacer trampa es $+8$
   * Si **no** estudié, la utilidad de no hacer trampa baja a $+3$

2. **Hacer trampa** ($d_1$):
   * **Profesor barco** ($\omega_0$) => $+10$ (probablemente "sale bien")
   * **Profesor normal** ($\omega_1$) => $+10$ (asumimos que, la mayoría de las veces, no te cacha)
   * **Profesor muy estricto** ($\omega_2$) => $-15$ (si te descubren)

Matriz de utilidad (si "estudié"=True):

| Estado | No trampa ($d_0$) | Trampa ($d_1$) |
|--------|-------------------|----------------|
| $\omega_0$ (barco) | $+8$ | $+10$ |
| $\omega_1$ (normal) | $+8$ | $+10$ |
| $\omega_2$ (estricto) | $+8$ | $-15$ |

Si "estudié"=False, se reemplaza el $+8$ por $+3$ en la columna de "no trampa".

### 2.6 Modelo Probabilístico

#### 2.6.1 A Priori Según la Universidad

**ITAM**:
* $p(\omega_0)=0.2$ (barco)
* $p(\omega_1)=0.3$ (normal)
* $p(\omega_2)=0.5$ (muy estricto)

**Ibero**:
* $p(\omega_0)=0.5$
* $p(\omega_1)=0.4$
* $p(\omega_2)=0.1$

#### 2.6.2 Probabilidad de Señales (proxy)

Definimos $p(z_1 \mid \omega)$, $p(z_2 \mid \omega)$ y $p(z_0 \mid \omega)$:

$z_1$ (relajado):
$$
p(z_1|\omega_0)=0.7,\quad p(z_1|\omega_1)=0.3,\quad p(z_1|\omega_2)=0.1
$$

$z_2$ (amenaza):
$$
p(z_2|\omega_0)=0.1,\quad p(z_2|\omega_1)=0.3,\quad p(z_2|\omega_2)=0.6
$$

$z_0$ (no hay señales claras):
$$
p(z_0|\omega_0)=0.2,\quad p(z_0|\omega_1)=0.4,\quad p(z_0|\omega_2)=0.3
$$

#### 2.6.3 Actualización de Probabilidades a Posteriori

Cuando observes una señal $z$, aplicas:

$$
p(\omega_i \mid z) = \frac{p(z \mid \omega_i)p(\omega_i)}{p(z)}
$$

donde

$$
p(z) = \sum_j p(z \mid \omega_j)p(\omega_j)
$$

## 3. Implementación y Ejemplo en Python

```python
import numpy as np

# 1. Distribuciones a priori en función de la universidad
def get_prior_probs(universidad):
    """
    Devuelve la distribución a priori de estados [p(barco), p(normal), p(estricto)]
    según la universidad.
    """
    if universidad == "ITAM":
        return np.array([0.2, 0.4, 0.4])  # Menos sesgado hacia estricto
    elif universidad == "Ibero":
        return np.array([0.6, 0.3, 0.1])  # Más probable ser barco
    else:
        return np.array([0.33, 0.34, 0.33])

# 2. Matriz de utilidad en base a si el alumno estudió o no
def get_utility_matrix(estudie):
    """
    Construye la matriz de utilidad con valores más balanceados
    """
    if estudie:
        u_no_trampa = 7  # Reducido de 8 a 7
    else:
        u_no_trampa = 3  # Se mantiene igual por ser bajo

    #        (no trampa, trampa)
    U = np.array([
        [u_no_trampa,  12],   # barco (aumentado beneficio)
        [u_no_trampa,  8],    # normal (reducido beneficio)
        [u_no_trampa, -20]    # muy estricto (aumentado castigo)
    ])
    return U

# 3. Probabilidades de las señales (más diferenciadas)
p_z1_given_omega = np.array([0.8, 0.2, 0.1])  # profesor relajado
p_z2_given_omega = np.array([0.1, 0.3, 0.8])  # profesor amenazante
p_z0_given_omega = np.array([0.1, 0.5, 0.1])  # sin señales claras

# 4. Función para calcular la posterior
def posterior_probs(prior, z):
    if z == 1:
        p_z_given_omega = p_z1_given_omega
    elif z == 2:
        p_z_given_omega = p_z2_given_omega
    else:
        p_z_given_omega = p_z0_given_omega

    p_z = np.sum(p_z_given_omega * prior)
    post = (p_z_given_omega * prior) / p_z
    return post

# 5. Cálculo de valor esperado de las decisiones
def valor_esperado(U, dist_estados):
    ve_no_trampa = np.sum(dist_estados * U[:,0])
    ve_trampa = np.sum(dist_estados * U[:,1])
    return (ve_no_trampa, ve_trampa)

# ========================= EJEMPLO DE USO =========================

def analizar_decision(uni, estudie):
    print(f"\n=== ANÁLISIS DE DECISIÓN ===")
    print(f"Universidad: {uni}, Estudié: {estudie}")
    
    # Obtén distribución inicial
    p_omega = get_prior_probs(uni)
    print("Probabilidades iniciales (barco, normal, estricto):", p_omega.round(3))
    
    # Construye matriz de utilidades
    U = get_utility_matrix(estudie)
    
    # Sin señales
    VE_sin_senal = valor_esperado(U, p_omega)
    print("\n--- Sin señales ---")
    print(f"VE(No trampa) = {VE_sin_senal[0]:.2f} | VE(Trampa) = {VE_sin_senal[1]:.2f}")
    print("=> Decisión sin señales:", "NO TRAMPA" if VE_sin_senal[0] > VE_sin_senal[1] else "HACER TRAMPA")
    
    # Con señal de profesor relajado
    post_z1 = posterior_probs(p_omega, 1)
    VE_z1 = valor_esperado(U, post_z1)
    print("\n--- Señal: Profesor relajado ---")
    print("Probabilidades actualizadas:", post_z1.round(3))
    print(f"VE(No trampa) = {VE_z1[0]:.2f} | VE(Trampa) = {VE_z1[1]:.2f}")
    print("=> Decisión con profesor relajado:", "NO TRAMPA" if VE_z1[0] > VE_z1[1] else "HACER TRAMPA")
    
    # Con señal de profesor amenazante
    post_z2 = posterior_probs(p_omega, 2)
    VE_z2 = valor_esperado(U, post_z2)
    print("\n--- Señal: Profesor amenazante ---")
    print("Probabilidades actualizadas:", post_z2.round(3))
    print(f"VE(No trampa) = {VE_z2[0]:.2f} | VE(Trampa) = {VE_z2[1]:.2f}")
    print("=> Decisión con profesor amenazante:", "NO TRAMPA" if VE_z2[0] > VE_z2[1] else "HACER TRAMPA")

# Analizar diferentes escenarios
print("================== ITAM - ESTUDIÓ ==================")
analizar_decision("ITAM", True)

print("\n================== ITAM - NO ESTUDIÓ ==================")
analizar_decision("ITAM", False)

print("\n================== IBERO - ESTUDIÓ ==================")
analizar_decision("Ibero", True)

print("\n================== IBERO - NO ESTUDIÓ ==================")
analizar_decision("Ibero", False)
```

## 4. Análisis de Resultados

1. **Sin señales ($z_0$)**:
   * Dependiendo de la **universidad** (ITAM vs. Ibero) y de si **estudiaste** o no, el **valor esperado** de "hacer trampa" o "no" cambiará
   * Por ejemplo, en el **ITAM**, con prob. alta de $\omega_2=0.5$, "hacer trampa" conlleva un riesgo mayor de $-15$. En cambio, si estudias y no haces trampa, tu utilidad es decente ($+8$)

2. **Con señal $z_1$ (profesor relajado)**:
   * Suben las probabilidades de $\omega_0$ y $\omega_1$, bajando la de $\omega_2$
   * Probablemente "hacer trampa" sea más rentable

3. **Con señal $z_2$ (profesor amenazó)**:
   * Sube la probabilidad de $\omega_2$, haciendo "no hacer trampa" más atractivo

4. **Efecto de "Estudié = True/False"**:
   * Si estudiaste, la utilidad de no hacer trampa sube ($+8$)
   * A veces, la **diferencia** entre no hacer trampa ($+8$) y hacer trampa ($+10$) no es tan grande; así, puede que no convenga arriesgarse a $-15$
   * Si **no estudiaste**, la ganancia base de no hacer trampa es solo $+3$, lo que hace más tentador arriesgarse a la trampa ($+10$ vs $+3$)

## 5. Heurísticas Alternativas

1. **Maximin**:
   * No consideras probabilidades, solo el **peor caso**
   * Peor caso de "no hacer trampa": $+3$ (si no estudiaste) o $+8$ (si estudiaste)
   * Peor caso de "hacer trampa": $-15$ (descubierto en $\omega_2$)
   * Gana "no hacer trampa" ($+3 > -15$)

2. **Regla de umbral** en $p(\omega_2)$:
   * "Si la $p(\omega_2) < 30\%$, hago trampa; si es $> 30\%$, no"
   * Se compara con el salto de utilidad

3. **Regla perezosa**:
   * "Si no estudié, me conviene la trampa para asegurar $+10$ (a menos que esté casi seguro que es estricto). Si sí estudié, mejor no arriesgar"

## 6. Conclusiones Finales

Este modelo **estático** de decisión bajo incertidumbre **incorpora** múltiples variables:

* **Universidad**: Afecta la **distribución** de los estados (barco, normal, estricto)
* **Haber estudiado**: Modifica la **utilidad** de no hacer trampa
* **Señal** (proxy) sobre la severidad del profesor: Actualiza (vía Bayes) la probabilidad de $\omega$

Así, tu **decisión óptima** (no trampa vs. trampa) dependerá de:
* Qué tan probable sea enfrentarte a un profesor muy estricto $\omega_2$
* La **diferencia** entre la ganancia de no hacer trampa (que es mayor si estudiaste) y la ganancia (o pérdida) de hacer trampa
* Las **actitudes** hacia el riesgo (puedes preferir un criterio maximin, un criterio de valor esperado, etc.)

Al final, con un **simple script en Python**, puedes **jugar** con distintos supuestos, señales y parámetros para **explorar** la toma de decisiones y entender mejor la influencia de cada factor. ¡Adelante con tus propias modificaciones y experimentos!

## Esto es para entenderlo mejor

Te explicaré este modelo de decisión bajo incertidumbre de una manera más intuitiva:

Imagina que estás por presentar un examen. Tienes dos opciones: hacer trampa o no hacerla. Parece simple, ¿verdad? Pero hay varios factores que complican esta decisión:

1. No sabes exactamente qué tipo de profesor te va a tocar:
   * Podría ser muy relajado ("barco")
   * Podría ser normal
   * Podría ser súper estricto

2. La universidad donde estás importa:
   * Si estás en el ITAM, hay más probabilidad de que te toque un profesor estricto (50%)
   * Si estás en la Ibero, es más probable que te toque uno relajado (50%)

3. Tu preparación también afecta:
   * Si estudiaste, puedes obtener un 8 sin hacer trampa (bastante decente)
   * Si no estudiaste, quizás solo saques un 3 sin trampa (no tan bueno)

4. Y lo más interesante: tienes "pistas" sobre el profesor:
   * ¿Mandó un correo amenazante sobre plagio? Mala señal
   * ¿Es súper relajado en clase? Buena señal
   * ¿No tienes idea? Neutral

Entonces, ¿cómo decides?

El modelo matemático que presentamos usa algo llamado "utilidad esperada". Básicamente:

* Si haces trampa:
  * Con profesor relajado o normal: +10 puntos (te sale bien)
  * Con profesor estricto: -15 puntos (te cachan y es desastre)

* Si no haces trampa:
  * Si estudiaste: +8 puntos siempre
  * Si no estudiaste: +3 puntos siempre

Lo interesante es cómo las "pistas" sobre el profesor afectan tu decisión. Por ejemplo:
* Si el profesor manda un correo super amenazante, la probabilidad de que sea estricto sube mucho
* Si en clase es súper relajado, es más probable que sea "barco"

El código en Python que incluimos te permite jugar con estos escenarios. Puedes ver cómo cambia tu decisión óptima según:
* La universidad donde estás
* Si estudiaste o no
* Las señales que recibes del profesor

Algunas conclusiones prácticas:
1. Si estudiaste, la diferencia entre hacer trampa (+10) y no hacerla (+8) es pequeña. ¿Vale la pena arriesgarse a un -15?
2. Si no estudiaste, la tentación es mayor porque la diferencia es más grande (+10 vs +3)
3. Las señales del profesor son importantes - te ayudan a actualizar tus probabilidades
4. La universidad importa - en lugares más estrictos, el riesgo de hacer trampa es mayor

En resumen, este modelo nos ayuda a entender cómo la incertidumbre (no saber qué tipo de profesor es) y la información parcial (las pistas que recibes) afectan una decisión aparentemente simple. Es un ejemplo perfecto de cómo las matemáticas pueden ayudarnos a analizar decisiones cotidianas de manera más estructurada.

¿Te gustaría que profundizáramos en algún aspecto específico del modelo?

In [2]:
import numpy as np

# 1. Distribuciones a priori en función de la universidad
def get_prior_probs(universidad):
    """
    Devuelve la distribución a priori de estados [p(barco), p(normal), p(estricto)]
    según la universidad.
    """
    if universidad == "ITAM":
        return np.array([0.2, 0.4, 0.4])  # Menos sesgado hacia estricto
    elif universidad == "Ibero":
        return np.array([0.6, 0.3, 0.1])  # Más probable ser barco
    else:
        return np.array([0.33, 0.34, 0.33])

# 2. Matriz de utilidad en base a si el alumno estudió o no
def get_utility_matrix(estudie):
    """
    Construye la matriz de utilidad con valores más balanceados
    """
    if estudie:
        u_no_trampa = 7  # Reducido de 8 a 7
    else:
        u_no_trampa = 3  # Se mantiene igual por ser bajo

    #        (no trampa, trampa)
    U = np.array([
        [u_no_trampa,  12],   # barco (aumentado beneficio)
        [u_no_trampa,  8],    # normal (reducido beneficio)
        [u_no_trampa, -20]    # muy estricto (aumentado castigo)
    ])
    return U

# 3. Probabilidades de las señales (más diferenciadas)
p_z1_given_omega = np.array([0.8, 0.2, 0.1])  # profesor relajado
p_z2_given_omega = np.array([0.1, 0.3, 0.8])  # profesor amenazante
p_z0_given_omega = np.array([0.1, 0.5, 0.1])  # sin señales claras

# 4. Función para calcular la posterior
def posterior_probs(prior, z):
    if z == 1:
        p_z_given_omega = p_z1_given_omega
    elif z == 2:
        p_z_given_omega = p_z2_given_omega
    else:
        p_z_given_omega = p_z0_given_omega

    p_z = np.sum(p_z_given_omega * prior)
    post = (p_z_given_omega * prior) / p_z
    return post

# 5. Cálculo de valor esperado de las decisiones
def valor_esperado(U, dist_estados):
    ve_no_trampa = np.sum(dist_estados * U[:,0])
    ve_trampa = np.sum(dist_estados * U[:,1])
    return (ve_no_trampa, ve_trampa)

# ========================= EJEMPLO DE USO =========================

def analizar_decision(uni, estudie):
    print(f"\n=== ANÁLISIS DE DECISIÓN ===")
    print(f"Universidad: {uni}, Estudié: {estudie}")
    
    # Obtén distribución inicial
    p_omega = get_prior_probs(uni)
    print("Probabilidades iniciales (barco, normal, estricto):", p_omega.round(3))
    
    # Construye matriz de utilidades
    U = get_utility_matrix(estudie)
    
    # Sin señales
    VE_sin_senal = valor_esperado(U, p_omega)
    print("\n--- Sin señales ---")
    print(f"VE(No trampa) = {VE_sin_senal[0]:.2f} | VE(Trampa) = {VE_sin_senal[1]:.2f}")
    print("=> Decisión sin señales:", "NO TRAMPA" if VE_sin_senal[0] > VE_sin_senal[1] else "HACER TRAMPA")
    
    # Con señal de profesor relajado
    post_z1 = posterior_probs(p_omega, 1)
    VE_z1 = valor_esperado(U, post_z1)
    print("\n--- Señal: Profesor relajado ---")
    print("Probabilidades actualizadas:", post_z1.round(3))
    print(f"VE(No trampa) = {VE_z1[0]:.2f} | VE(Trampa) = {VE_z1[1]:.2f}")
    print("=> Decisión con profesor relajado:", "NO TRAMPA" if VE_z1[0] > VE_z1[1] else "HACER TRAMPA")
    
    # Con señal de profesor amenazante
    post_z2 = posterior_probs(p_omega, 2)
    VE_z2 = valor_esperado(U, post_z2)
    print("\n--- Señal: Profesor amenazante ---")
    print("Probabilidades actualizadas:", post_z2.round(3))
    print(f"VE(No trampa) = {VE_z2[0]:.2f} | VE(Trampa) = {VE_z2[1]:.2f}")
    print("=> Decisión con profesor amenazante:", "NO TRAMPA" if VE_z2[0] > VE_z2[1] else "HACER TRAMPA")

# Analizar diferentes escenarios
print("================== ITAM - ESTUDIÓ ==================")
analizar_decision("ITAM", True)

print("\n================== ITAM - NO ESTUDIÓ ==================")
analizar_decision("ITAM", False)

print("\n================== IBERO - ESTUDIÓ ==================")
analizar_decision("Ibero", True)

print("\n================== IBERO - NO ESTUDIÓ ==================")
analizar_decision("Ibero", False)

================== ITAM - ESTUDIÓ ==================

=== ANÁLISIS DE DECISIÓN ===
Universidad: ITAM, Estudié: True
Probabilidades iniciales (barco, normal, estricto): [0.2 0.4 0.4]

--- Sin señales ---
VE(No trampa) = 7.00 | VE(Trampa) = -2.40
=> Decisión sin señales: NO TRAMPA

--- Señal: Profesor relajado ---
Probabilidades actualizadas: [0.571 0.286 0.143]
VE(No trampa) = 7.00 | VE(Trampa) = 6.29
=> Decisión con profesor relajado: NO TRAMPA

--- Señal: Profesor amenazante ---
Probabilidades actualizadas: [0.043 0.261 0.696]
VE(No trampa) = 7.00 | VE(Trampa) = -11.30
=> Decisión con profesor amenazante: NO TRAMPA

================== ITAM - NO ESTUDIÓ ==================

=== ANÁLISIS DE DECISIÓN ===
Universidad: ITAM, Estudié: False
Probabilidades iniciales (barco, normal, estricto): [0.2 0.4 0.4]

--- Sin señales ---
VE(No trampa) = 3.00 | VE(Trampa) = -2.40
=> Decisión sin señales: NO TRAMPA

--- Señal: Profesor relajado ---
Probabilidades actualizadas: [0.571 0.286 0.143]
VE(No tr

# Análisis de Decisión Bayesiano: Proceso Paso a Paso

## Definiciones Iniciales

Sea $\Omega$ el conjunto de estados posibles del profesor:
* $\omega_1$: Profesor barco
* $\omega_2$: Profesor normal
* $\omega_3$: Profesor estricto

Sea $D$ el conjunto de decisiones posibles:
* $d_0$: No hacer trampa
* $d_1$: Hacer trampa

Sea $Z$ el conjunto de señales observables:
* $z_0$: Sin señales claras
* $z_1$: Profesor parece relajado
* $z_2$: Profesor parece estricto

## Proceso de Decisión

### 1. Distribución A Priori $P(\omega)$

Dependiendo de la universidad, asignamos probabilidades iniciales:

**ITAM**:
$$P(\omega) = [0.2, 0.4, 0.4]$$

**Ibero**:
$$P(\omega) = [0.6, 0.3, 0.1]$$

### 2. Matriz de Utilidad $U(\omega,d)$

Si el alumno estudió:
$$U = \begin{pmatrix} 
7 & 12 \\
7 & 8 \\
7 & -20
\end{pmatrix}$$

Si el alumno no estudió:
$$U = \begin{pmatrix}
3 & 12 \\
3 & 8 \\
3 & -20
\end{pmatrix}$$

donde:
* Filas: estados $\omega$ (barco, normal, estricto)
* Columnas: decisiones $d$ (no trampa, trampa)

### 3. Probabilidades Condicionales de Señales $P(z|\omega)$

$$P(z_1|\omega) = [0.8, 0.2, 0.1]$$ 
$$P(z_2|\omega) = [0.1, 0.3, 0.8]$$
$$P(z_0|\omega) = [0.1, 0.5, 0.1]$$

### 4. Actualización Bayesiana

Para una señal observada $z$, calculamos:

$$P(\omega|z) = \frac{P(z|\omega)P(\omega)}{P(z)}$$

donde:
$$P(z) = \sum_j P(z|\omega_j)P(\omega_j)$$

### 5. Valor Esperado

Para cada decisión $d$, calculamos:

$$VE(d) = \sum_i P(\omega_i)U(\omega_i,d)$$

## Ejemplo Numérico: ITAM con Estudiante que Estudió

### A. Sin Señales

1. Prior: $P(\omega) = [0.2, 0.4, 0.4]$

2. $VE(d_0)$ = No Trampa:
   $$VE(d_0) = 0.2(7) + 0.4(7) + 0.4(7) = 7$$

3. $VE(d_1)$ = Trampa:
   $$VE(d_1) = 0.2(12) + 0.4(8) + 0.4(-20) = -2.4$$

4. Decisión: No hacer trampa pues $7 > -2.4$

### B. Con Señal $z_1$ (Profesor Relajado)

1. Calculamos $P(\omega|z_1)$:
   * $P(z_1) = 0.8(0.2) + 0.2(0.4) + 0.1(0.4) = 0.28$
   * $P(\omega|z_1) = [0.57, 0.29, 0.14]$

2. Nuevos valores esperados:
   * $VE(d_0) = 0.57(7) + 0.29(7) + 0.14(7) = 7$
   * $VE(d_1) = 0.57(12) + 0.29(8) + 0.14(-20) = 6.36$

3. Decisión: No hacer trampa pues $7 > 6.36$

## Conclusión

La decisión óptima es aquella que maximiza el valor esperado:

$$d^* = \arg\max_d \sum_i P(\omega_i)U(\omega_i,d)$$

donde $P(\omega_i)$ puede ser:
* La distribución a priori $P(\omega)$
* La distribución posterior $P(\omega|z)$ si observamos una señal